In [11]:
!which python

/home/ec2-user/anaconda3/envs/python3/bin/python


In [12]:
import boto3

In [13]:
s3=boto3.client("s3")

In [14]:
response = s3.list_objects_v2(Bucket="llm-finetune-dataset-sunny", Prefix="datasets/")

In [15]:
for obj in response.get("Contents", []):
    print(obj)

{'Key': 'datasets/', 'LastModified': datetime.datetime(2025, 11, 9, 9, 41, 6, tzinfo=tzlocal()), 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 0, 'StorageClass': 'STANDARD'}
{'Key': 'datasets/pharma_instruction_data.csv', 'LastModified': datetime.datetime(2025, 11, 9, 10, 35, 6, tzinfo=tzlocal()), 'ETag': '"7a3bbfb7acae4d8dc88dc1a08501d745"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 2469, 'StorageClass': 'STANDARD'}


In [16]:
for obj in response.get("Contents", []):
    print(obj["Key"])

datasets/
datasets/pharma_instruction_data.csv


In [17]:
dataset_path = "s3://llm-finetune-dataset-sunny/datasets/pharma_instruction_data.csv/"

In [18]:
from datasets import load_dataset

In [35]:
dataset = load_dataset("csv",data_files={"train":dataset_path},split="train")
# dataset = load_dataset("csv", data_files="/content/pharma_instruction_data.csv",split="train")

In [36]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})

In [38]:
print(dataset)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5
})


In [40]:
print(dataset[0])

{'instruction': 'Explain the mechanism of action of Metformin.', 'input': None, 'output': 'Metformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'}


In [41]:
def format_example(example):
    prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}"
    return {"text": prompt}

In [42]:
dataset = dataset.map(format_example)

In [43]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 5
})

In [44]:
dataset['text'][0]

'### Instruction:\nExplain the mechanism of action of Metformin.\n### Input:\nNone\n### Response:\nMetformin activates AMP-activated protein kinase (AMPK), which increases glucose uptake and fatty-acid oxidation while inhibiting hepatic gluconeogenesis, thereby lowering blood glucose.'

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [51]:
model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

In [52]:
tokenizer = AutoTokenizer.from_pretrained(model)

In [53]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [54]:
def tokenize_fn(example):
    tokens = tokenizer(example["text"], truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

In [55]:
tokenized = dataset.map(tokenize_fn, batched=True)

In [56]:
from peft import LoraConfig, get_peft_model, TaskType

In [57]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

RuntimeError: unable to mmap 4400216536 bytes from file </home/ec2-user/.cache/huggingface/hub/models--TinyLlama--TinyLlama-1.1B-intermediate-step-1431k-3T/snapshots/59f6f375b26bde864a6ca194a9a3044570490064/model.safetensors>: Cannot allocate memory (12)

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none"
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
args = TrainingArguments(
    output_dir="./tinyllama-instruction",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none"
)

In [ ]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized,
)

In [ ]:
trainer.train()

In [ ]:

model_path = "/content/tinyllama-instruction/checkpoint-3"

In [ ]:

instruction_model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

In [ ]:

prompt = "Explain the mechanism of action of Metformin."

In [ ]:

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

In [ ]:

outputs = instruction_model.generate(
    **inputs,
    max_new_tokens=100,
    temperature=0.8,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1
)

In [ ]:

print("\nModel Output:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))